In [ ]:
# Importing the required libraries
import pandas  as pd
import numpy as np
import matplotlib.pyplot  as plt
import cv2

import tensorflow as tf 
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D

In [ ]:
train_csv_path = "../input/cassava-leaf-disease-classification/train.csv"
label_json_path = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
images_dir_path = "../input/cassava-leaf-disease-classification/train_images"

In [ ]:
train_csv = pd.read_csv(train_csv_path)
train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_path, orient='index')
label_class = label_class.values.flatten().tolist()

In [ ]:
IMG_SIZE = 288
BATCH_SIZE = 12
EPOCHS = 32//16
lr = 1e-5

# **Data Agumentation and Pre-Processing**

In [ ]:
# Data agumentation and pre-processing using tensorflow# Data agumentation and pre-processing using tensorflow
train_gen = ImageDataGenerator(
                                rotation_range=270,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                brightness_range=[0.1,0.9],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.2
                               )
                                    
    
valid_gen = ImageDataGenerator(rescale=1/255,
                               validation_split = 0.2
                              )

In [ ]:
train_generator = train_gen.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            subset = "training",

)

valid_generator = valid_gen.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            subset = "validation"
)

In [ ]:
PRECISION = tf.keras.metrics.Precision()
RECALL = tf.keras.metrics.Recall()

def F1_score(y_true, y_pred):
    
    PRECISION.reset_states()
    RECALL.reset_states()

    precision_out = PRECISION.update_state(y_true, y_pred)
    precision_out = PRECISION.result()

    recall_out = RECALL.update_state(y_true, y_pred)
    recall_out = RECALL.result()

    return 2 * ((precision_out*recall_out)/(precision_out+recall_out+1e-23))

In [ ]:
BASE0 = applications.MobileNet(include_top=False, 
                                    input_shape=[IMG_SIZE, IMG_SIZE, 3], weights=None,
                                    pooling='max')

def build_model(input_size = [IMG_SIZE, IMG_SIZE, 3]):
    model = tf.keras.Sequential()
    model.add(BASE0)
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9),
                  metrics=['accuracy', F1_score])
    
    return model

In [ ]:
#model0 = tf.keras.models.load_model("../input/cassavaleafdiseasemodels/CasavaLeafDiseaseModel_acc_86_12.h5", custom_objects={"F1_score":F1_score})
#model1 = tf.keras.models.load_model("../input/cassavaleafdiseasemodels/CasavaLeafDiseaseModel_sgd_opt_acc_85_9.h5", custom_objects={"F1_score":F1_score})

In [ ]:
# A callback to save the model
callback0 = tf.keras.callbacks.ModelCheckpoint("model.h5", monitor='val_loss',save_best_only=True)

In [ ]:
his = model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS, callbacks=[callback0])

In [ ]:
model.save('model.h5', include_optimizer=False)

In [ ]:
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=True) 

In [ ]:
# Submission file ouput
print("Submission File: \n---------------\n")
print(my_submission.head()) # Predicted Output